In [97]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.utils.data as Data
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib import image as mp_image
import seaborn as sns
import torch.optim as optim
import torch.nn.functional as F

In [99]:
device = "cuda" if torch.cuda.is_available else "cpu"
device = "cpu"
print(device)

# seed
np.random.seed(777)
torch.manual_seed(777)

# GPU seed
if device == "cuda":
    torch.cuda.manual_seed_all(777)

cpu


In [101]:
learning_rate = 0.001
training_epoch = 10
batch_size = 100

In [103]:
def make_file_list(path):
    train_img_list = list()

    with os.scandir(path) as files:
        for file in files:
            if file.name.endswith('.jpg'):
              # adds only the image files to the flowers list
                train_img_list.append(f"{path}/{file.name}")
    
    return train_img_list

In [105]:
# make train dataloader list

hot_dogs_path = r"./archive/train/hot_dog"
not_hot_dogs_path = r"./archive/train/not_hot_dog"

hot_dogs_train = make_file_list(hot_dogs_path)
not_hot_dogs_train = make_file_list(not_hot_dogs_path)

hot_dogs_train[0]

'./archive/train/hot_dog/3622018.jpg'

In [106]:
# image transform 
class ImageTransform():
    def __init__(self, mean, std):
        self.data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
        
    def __call__(self, img):
        return self.data_transform(img)

In [107]:
from PIL import Image

# Custom Dataset
class ImageDataset(Data.Dataset):
    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform
    
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        return img_transformed

In [171]:
trans = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
#     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

training_path = r"./archive/train"
test_path = r"./archive/test"
train_data = datasets.ImageFolder(root=training_path, transform=trans)
test_data = datasets.ImageFolder(root=test_path, transform=trans)


train_set = torch.utils.data.DataLoader(dataset=train_data, batch_size=30, shuffle=True)
test_set = torch.utils.data.DataLoader(dataset=test_data)

In [172]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = nn.Flatten()(out)
        out = self.fc(out)
        return out


In [188]:
# Create a neural net class
class Net(nn.Module):
    
    
    # Defining the Constructor
    def __init__(self, num_classes=2):
        super(Net, self).__init__()
        
        # In the init function, we define each layer we will use in our model
        
        # Our images are RGB, so we have input channels = 3. 
        # We will apply 12 filters in the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # A second convolutional layer takes 12 input channels, and generates 24 outputs
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # We in the end apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # This means that our feature tensors are now 32 x 32, and we've generated 24 of them
        
        # We need to flatten these in order to feed them to a fully-connected layer
        self.fc = nn.Linear(in_features=64 * 64 * 24, out_features=8*8*24)
        self.fc2 = nn.Linear(in_features=8*8*24, out_features=num_classes, bais=True)

    def forward(self, x):
        # In the forward function, pass the data through the layers we defined in the init function
        
        # Use a ReLU activation function after layer 1 (convolution 1 and pool)
        x = F.relu(self.pool(self.conv1(x))) 
        
        # Use a ReLU activation function after layer 2
        x = F.relu(self.pool(self.conv2(x)))  
        
        # Select some features to drop to prevent overfitting (only drop during training)
        x = F.dropout(self.drop(x), training=self.training)
        
        # Flatten
        x  = nn.Flatten()(x)
        
        # Feed to fully-connected layer to predict class
        x = self.fc(x)
        x = self.fc2(x)
        # Return class probabilities via a log_softmax function 
        return x
    

In [189]:
model = Net().to(device)

TypeError: __init__() got an unexpected keyword argument 'bais'

In [190]:
def train(model, device, train_loader, optimizer, epoch):
    # Set the model to training mode
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):
        # Use the CPU or GPU as appropriate
        # Recall that GPU is optimized for the operations we are dealing with
        data, target = data.to(device), target.to(device)
        
        # Reset the optimizer
        optimizer.zero_grad()
        
        # Push the data forward through the model layers
        output = model(data)
        
        # Get the loss
        loss = loss_criteria(output, target)

        # Keep a running total
        train_loss += loss.item()
        
        # Backpropagate
        loss.backward()
        optimizer.step()
        
        predicted = torch.argmax(output, 1) == target
        running_correct = predicted.float().mean()
        
        # Print metrics so we see some progress
        print('\tTraining batch {} Loss: {:.6f} cost = {:.4f}'.format(batch_idx + 1, loss.item(),running_correct*100))
            
    # return average loss for the epoch
    avg_loss = train_loss / (batch_idx+1)
    print('Training set: Average loss: {:.6f} '.format(avg_loss))
    return avg_loss

In [191]:
def test(model, device, test_loader):
    # Switch the model to evaluation mode (so we don't backpropagate or drop)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for data, target in test_loader:
            batch_count += 1
            data, target = data.to(device), target.to(device)
            
            # Get the predicted classes for this batch
            output = model(data)
            
            # Calculate the loss for this batch
            test_loss += loss_criteria(output, target).item()
            
            # Calculate the accuracy for this batch
            _, predicted = torch.max(output.data, 1)
            correct += torch.sum(target==predicted).item()

    # Calculate the average loss and total accuracy for this epoch
    avg_loss = test_loss / batch_count
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # return average loss for the epoch
    return avg_loss

In [ ]:
# Use an "Adam" optimizer to adjust weights
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the loss criteria
loss_criteria = nn.CrossEntropyLoss()

# Track metrics in these arrays
epoch_nums = []
training_loss = []


# Train over 10 epochs (We restrict to 10 for time issues)
epochs = 10
print('Training on', device)
for epoch in range(1, epochs + 1):
        train_loss = train(model, device, train_set, optimizer, epoch)
        test_loss = test(model, device, test_set)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)

Training on cpu
Epoch: 1
	Training batch 1 Loss: 0.095566 cost = 100.0000
	Training batch 2 Loss: 0.561506 cost = 70.0000
	Training batch 3 Loss: 0.073822 cost = 96.6667
	Training batch 4 Loss: 0.436284 cost = 76.6667
	Training batch 5 Loss: 0.262155 cost = 90.0000
	Training batch 6 Loss: 0.097429 cost = 96.6667
	Training batch 7 Loss: 0.201020 cost = 93.3333
	Training batch 8 Loss: 0.417563 cost = 76.6667
	Training batch 9 Loss: 0.361313 cost = 90.0000
	Training batch 10 Loss: 0.081345 cost = 100.0000
	Training batch 11 Loss: 0.191327 cost = 93.3333
	Training batch 12 Loss: 0.143621 cost = 96.6667
	Training batch 13 Loss: 0.199051 cost = 93.3333
	Training batch 14 Loss: 0.135661 cost = 96.6667
	Training batch 15 Loss: 0.139023 cost = 90.0000
	Training batch 16 Loss: 0.118279 cost = 93.3333
	Training batch 17 Loss: 0.168247 cost = 94.4444
Training set: Average loss: 0.216659 
Validation set: Average loss: 1.014064, Accuracy: 288/500 (58%)

Epoch: 2
	Training batch 1 Loss: 0.166987 cost

In [185]:
validation_loss = []


test_loss = test(model, device, test_set)
validation_loss.append(test_loss)

Validation set: Average loss: 0.925093, Accuracy: 286/500 (57%)



In [186]:
# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# 옵티마이저의 state_dict 출력
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([12, 3, 3, 3])
conv1.bias 	 torch.Size([12])
conv2.weight 	 torch.Size([24, 12, 3, 3])
conv2.bias 	 torch.Size([24])
fc.weight 	 torch.Size([2, 98304])
fc.bias 	 torch.Size([2])
Optimizer's state_dict:
state 	 {0: {'step': 170, 'exp_avg': tensor([[[[ 0.0158,  0.0135,  0.0103],
          [ 0.0145,  0.0129,  0.0094],
          [ 0.0111,  0.0084,  0.0063]],

         [[ 0.0338,  0.0305,  0.0265],
          [ 0.0327,  0.0304,  0.0257],
          [ 0.0286,  0.0253,  0.0222]],

         [[ 0.0368,  0.0324,  0.0274],
          [ 0.0359,  0.0325,  0.0273],
          [ 0.0316,  0.0274,  0.0241]]],


        [[[ 0.0079,  0.0087,  0.0100],
          [ 0.0060,  0.0074,  0.0082],
          [ 0.0004,  0.0026,  0.0033]],

         [[ 0.0137,  0.0141,  0.0146],
          [ 0.0106,  0.0116,  0.0117],
          [ 0.0032,  0.0049,  0.0050]],

         [[ 0.0124,  0.0118,  0.0116],
          [ 0.0094,  0.0097,  0.0094],
          [ 0.0021,  0.0033,  0.0031]]],

In [187]:
torch.save(model, 'model2.pt')